# DeepBox

## First we need to import a few things, this includes our generator and visualizing module

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os
from Deep_Learning.Base_Deeplearning_Code.Visualizing_Model.Visualing_Model import visualization_model_class
%matplotlib inline

## What do we need? We need a way to generate larges amounts of training data for our model..

In [ ]:
from Deep_Learning.Shape_Maker import Data_Generator, make_rectangle, make_circle
image_size = 64

## The make_rectangle and make_circle will both return circles and rectangles, and the Data_Generator will randomly create circles or rectangles

In [ ]:
plt.imshow(make_rectangle(image_size))

In [ ]:
plt.imshow(make_circle(image_size))

### Our generator essentially continiously creates examples

In [ ]:
train_generator = Data_Generator(image_size=image_size,batch_size=32, num_examples_per_epoch=150)

In [ ]:
x,y = train_generator.__getitem__(0)
print(x.shape)
print(y.shape)

## Now lets make our network!

In [ ]:
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Activation
from tensorflow.python.keras.optimizers import Adam
import tensorflow.compat.v1 as tf
import tensorflow.python.keras.backend as K
from tensorflow.compat.v1 import Graph, Session, ConfigProto, GPUOptions

### This will make sure multiple networks don't clog up the GPU

In [ ]:
def prep_network():
    K.clear_session()
    gpu_options = tf.GPUOptions(allow_growth=True)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    K.set_session(sess)
    return None

### Representation of network

![DeepBox_Network.png](./Deep_Learning/DeepBox_Network.png)

### Building the network

In [ ]:
prep_network()
num_kernels = 4
kernel_size = (3,3)
model = Sequential([
    Conv2D(num_kernels, kernel_size, 
           input_shape=(image_size, image_size, 1), 
           padding='same',name='Conv_0',activation='sigmoid'),
    MaxPool2D((image_size)), # Pool into a 1x1x4 image
    Flatten(),
    Dense(2,activation='softmax')
])

### Defining loss
#### We are specifying that we care about the categorical cross entropy, with a learning rate of 0.1 (very high)

In [ ]:
model.compile(Adam(lr=1e-1), loss='categorical_crossentropy', metrics=['accuracy'])

### Train
#### We give the model our generator, and tell it to run for 5 epochs

In [ ]:
model.fit_generator(train_generator,epochs=5)

### Evaluate
### We will randomly create 500 examples of rectangles and circles and see how well our model does

In [ ]:
def determine_accuracy(model, image_size= 64, num_examples=1000):
    truth = np.zeros((num_examples,1))
    guess = np.zeros((num_examples,1))
    index = 0
    for _ in range(num_examples//2):
        pred = model.predict(make_rectangle(image_size)[None,...,None])
        guess[index] = np.argmax(pred)
        truth[index] = 1
        index += 1
    for _ in range(num_examples//2):
        pred = model.predict(make_circle(image_size)[None,...,None])
        guess[index] = np.argmax(pred)
        index += 1
    print('Accuracy is {} for {} examples'.format(str((guess==truth).sum()/num_examples),num_examples))

In [ ]:
determine_accuracy(model)

### Lets see how confident it is in it's predictions, generate a random circle or rectangle and see what the confidence is

In [ ]:
rectangle = make_rectangle(image_size)[None,...,None]
circle = make_circle(image_size)[None,...,None]
print('{}% confident'.format(model.predict(rectangle)[...,1][0]*100))
print('{}% confident'.format(model.predict(circle)[...,0][0]*100))

## Lets see what the kernels and activations look like

In [ ]:
Visualizing_Class = visualization_model_class(model=model)

### Say that we only want to look at Conv_0

In [ ]:
Visualizing_Class.define_desired_layers(desired_layer_names=['Conv_0'])

## Kernels

In [ ]:
Visualizing_Class.plot_kernels()

## Activations
### In order to make an activation map we need to provide it with something to predict on

In [ ]:
Visualizing_Class.predict_on_tensor(make_rectangle(image_size)[None,...,None])

In [ ]:
Visualizing_Class.plot_activations()

## How big is this model? Super tiny!!

In [ ]:
model.summary()